In [ ]:
import geopandas
import intake

M2_TO_ACRE = 4046.86

In [ ]:
cat = intake.open_catalog("sources.yaml")

## VCM Data

We emailed Avery Hill, the first author of
[Low-elevation conifers in California’s Sierra Nevada are out of equilibrium with climate](https://doi.org/10.1093/pnasnexus/pgad004),
and received a copy of the vegetation climat mismatch (VCM) data used in their study. These data are
licensed under a CC-BY-SA license.

We looked at future projections of VCM under SSP245, a moderately ambitious climate mitigation
pathway. Those data were provided in vector format. We load the VCM data and then intersect it with
project boundary data.


In [ ]:
def transform_vcm_data(data):
    """repoject and set index of Hill et al., VCM data"""

    crs = "EPSG:26710"

    data = data.to_crs(crs)
    data = data.set_index("Conifer_Classification")
    return data


vcm_data = {
    "current": transform_vcm_data(cat["current_vcm"].read()),
    "future": transform_vcm_data(cat["future_vcm"].read()),
}

# Project Boundary data

We downloaded project boundary data from
[the Climate Action Reserve registry](https://thereserve2.apx.com/).

Sierra Pacific initially listed several of the boundaries we examined as separate projects. As the
projects moved through development, SPI combined several of these projects together. For instance, a
note attached to project the Martell project (CAR1409) reads: "Project combined with CAR1386,
CAR1404, CAR1405, and CAR1406." The RBS 2019 project (CAR1381) has a similar history.

Working with provisional project boundary data has practical implications for our analysis, because
the final project boundaries (and therefore their intersection with the VCM data) might be different
from the data we have available. Furthermore, we have strong evidence that the data we have is
incomplete. According to paperwork for CAR1409, total project acreage is reported at 87,266 acres.
The total area of the four shapefiles we downloaded, however, is only 75,823 acres. In this case, it
appears we do not have access to project boundary data for a fifth project area named "Blue." We
account for this missing data in our analysis by making the maximally conservative assumption that
100% of the unaccounted for acreage falls within the future habitable zone of confiers in the Sierra
Nevada.

Missingness also occurs in cases where a project has acreage that falls outside the study reigon of
Hill et al., 2023. RBS 2019, for example, has at least two properties that have no intersection with
the provided VCM data. Less than 20 percent of RBS 2019's acreage falls within the zombie forest
study area. In these cases, we again make a maximally conservative assumption, assuming that confier
habit will remain stable in all regions where the VCM metric was not explicitly calculated.

For completeness, we should also note that there is a fifth project named Crane Valley (CAR1114)
that also falls within the zombie forest study area. Like the other projects examined in this
notebook, it is also owned by SPI. However, we excluded it from our analysis because the project
appears to have been dormant since 2017 despite having mostly complete paperwork.


In [ ]:
projects = {
    "rbs": {
        "ninemile": "CAR1378",
        "bull_creek": "CAR1379",
        "big_bend": "CAR1380",
        "rbs": "CAR1381",
    },
    "martell": {
        "tiger": "CAR1386",
        "schaads": "CAR1405",
        "swamp": "CAR1406",
        "westel": "CAR1404",
    },
    "mosquito": {"mosquito": "CAR1384"},
    "cohasset": {"cohasset": "CAR1382"},
}

In [ ]:
store = {}
for label, d in projects.items():
    project_gdf = geopandas.pd.concat(
        [cat.boundary_data(opr_id=opr_id).read() for opr_id in d.values()]
    ).dissolve()
    project_area = (project_gdf.area / M2_TO_ACRE).iloc[0]
    if label == "martell":
        project_area = 87_266  # from `ARB_SPI_Martell_RP1_VerificationReport_V1-3_060322.pdf`
    acres_per_class = {
        k: geopandas.clip(vcm_gdf, project_gdf).area / M2_TO_ACRE for k, vcm_gdf in vcm_data.items()
    }
    print()
    frac_vulnerable = {
        k: vcm_acres.filter(like="VCM").sum() / project_area
        for k, vcm_acres in acres_per_class.items()
    }
    frac_examined = acres_per_class["future"].sum() / project_area

    print(
        f"{frac_examined * 100:.2f}% of {label} acreage falls within Hill et al. 2023 study region"
    )
    store[label] = frac_vulnerable
display(store)

# Reforestation project

We identified one reforestation project within the study area, the Camp refo (CAR1491).
Reforestation of the region with confier species is especially risky, as pointed out by Hill et al.,
2023:

> Most notably, in a VCM area, efforts to reforest after a fire or other disturbance with the same
> vegetation type as before are unlikely to be successful. Post-disturbance restoration needs to
> take into account the species mix and density that can currently be supported, but also the kinds
> of vegetation that future conditions are likely to support. This requires managers grappling with
> uncertainty in climate projections as they plan the future of the lands they manage; accessible
> tools that can synthesize climate projections with species distribution modeling can facilitate
> this planning process.

This particular project plans to replant conifer species in a region that is projected to be
severely mismatch with the habitat needs of those species.


In [ ]:
# have to read raw file, as can't quite figure out how to pass kwargs to intake_geopandas plugin
camp_refor = cat.boundary_data(opr_id="CAR1491", geopandas_kwargs={"layer": "Project_Area"}).read()
camp_acres = geopandas.clip(vcm_data["future"], camp_refor).area / M2_TO_ACRE

total_acres = max(
    camp_acres.sum(), 12_713
)  # must be at least as large as listing doc acreage of 12,713
display(camp_acres / total_acres)

### Crediting

Many of the projects have been issued registry offset credits. However, those credits have yet to be
converted in ARB offset credits, as none of the projects discussed here yet appear in the official
ARB
[issuance table](https://ww2.arb.ca.gov/our-work/programs/compliance-offset-program/arb-offset-credit-issuance)


In [ ]:
crediting = {
    "rbs": 1_925_806,
    "martell": 293_284,
    "mosquito": 366_335,
    "cohasset": 97_189,
}
display(sum(crediting.values()))